### Imports

In [1]:
# Remove unwanted warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

# Data Management
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import date as date
from ta import add_all_ta_features
from utils.my_functions import unify_column_names

# Statistics
from statsmodels.tsa.stattools import adfuller

# Unsupervised Machine Learning
from sklearn.decomposition import PCA

# Supervised Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

# Reporting
import matplotlib.pyplot as plt

### Initial Data Extraction

In [2]:
# Data Extraction
start_date = '2017-01-01'
end_date = date.today()
symbol = "VIX"

# yfinance data:
data = yf.download(symbol, start_date, end_date, auto_adjust=True, progress=False)
data

Price,Close,High,Low,Open,Volume
Ticker,VIX,VIX,VIX,VIX,VIX
Date,,,,,
2017-01-03,5238.100098,5238.100098,4761.899902,4952.379883,217416
2017-01-04,5238.100098,5333.330078,5142.859863,5142.859863,54528
2017-01-05,5238.100098,5238.100098,5047.620117,5142.859863,38974
2017-01-06,5047.620117,5238.100098,5047.620117,5142.859863,102008
2017-01-09,5047.620117,5142.859863,4952.379883,4952.379883,129879
...,...,...,...,...,...
2018-01-22,9800.000000,10000.000000,9600.000000,9900.000000,257523
2018-01-23,9900.000000,9900.000000,9700.000000,9900.000000,184915


In [3]:
data.head()

Price,Close,High,Low,Open,Volume
Ticker,VIX,VIX,VIX,VIX,VIX
Date,,,,,
2017-01-03,5238.100098,5238.100098,4761.899902,4952.379883,217416
2017-01-04,5238.100098,5333.330078,5142.859863,5142.859863,54528
2017-01-05,5238.100098,5238.100098,5047.620117,5142.859863,38974
2017-01-06,5047.620117,5238.100098,5047.620117,5142.859863,102008
2017-01-09,5047.620117,5142.859863,4952.379883,4952.379883,129879


In [4]:
data.tail()

Price,Close,High,Low,Open,Volume
Ticker,VIX,VIX,VIX,VIX,VIX
Date,,,,,
2018-01-22,9800.0,10000.0,9600.0,9900.0,257523
2018-01-23,9900.0,9900.0,9700.0,9900.0,184915
2018-01-26,9300.0,9500.0,8400.0,9300.0,277990
2018-01-30,9300.0,9700.0,9300.0,9700.0,311049
2018-01-31,9300.0,9400.0,9100.0,9400.0,82483


In [5]:
data.describe()

Price,Close,High,Low,Open,Volume
Ticker,VIX,VIX,VIX,VIX,VIX
count,266.000000,266.000000,266.000000,266.000000,2.660000e+02
mean,7281.489698,7431.382190,7099.409512,7247.923635,6.246855e+05
std,1667.977176,1735.056642,1545.531577,1650.170048,6.564761e+05
min,4952.379883,5142.859863,4761.899902,4952.379883,0.000000e+00
25%,6023.810059,6095.240234,6000.000000,6000.000000,1.773428e+05
50%,7000.000000,7200.000000,7000.000000,7050.000000,3.791645e+05
75%,7600.000000,7700.000000,7400.000000,7500.000000,8.057585e+05
max,12800.000000,13000.000000,11700.000000,12900.000000,3.143645e+06


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 266 entries, 2017-01-03 to 2018-01-31
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   (Close, VIX)   266 non-null    float64
 1   (High, VIX)    266 non-null    float64
 2   (Low, VIX)     266 non-null    float64
 3   (Open, VIX)    266 non-null    float64
 4   (Volume, VIX)  266 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 12.5 KB


In [7]:
# Apply transformation to column names
data.columns = data.columns.map(unify_column_names)

In [8]:
# Flatten the multiindex columns
data.columns = data.columns.get_level_values(0)

In [9]:
# Add TA to data
data = add_all_ta_features(data, open="open", high="high", low="low", close="close", volume="volume", fillna=True)
data

Price,close,high,low,open,volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-03,5238.100098,5238.100098,4761.899902,4952.379883,217416,2.174160e+05,217416,1.000000,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5238.100098,0.000000,0.000000,0.000000
2017-01-04,5238.100098,5333.330078,5142.859863,5142.859863,54528,2.174189e+05,271944,0.799499,0.000000e+00,8.316828e+07,...,0.000000,0.000000,0.000000,-6.327685,-1.265537,-5.062148,5238.100098,0.000000,0.000000,0.000000
2017-01-05,5238.100098,5238.100098,5047.620117,5142.859863,38974,2.563929e+05,310918,0.824632,0.000000e+00,-4.654471e+07,...,0.000000,0.000000,0.000000,-12.571892,-3.526808,-9.045084,5238.100098,0.000000,0.000000,0.000000
2017-01-06,5047.620117,5238.100098,5047.620117,5142.859863,102008,1.543849e+05,208910,0.373880,-2.775783e+06,0.000000e+00,...,-0.290869,-0.058174,-0.232695,-14.920413,-5.805529,-9.114884,5138.445852,-3.636433,-3.704199,-3.636433
2017-01-09,5047.620117,5142.859863,4952.379883,4952.379883,129879,1.543853e+05,338789,0.284421,-2.379243e+06,-1.396789e+07,...,-0.516736,-0.149886,-0.366850,-15.385465,-7.721516,-7.663949,5087.134260,0.000000,0.000000,-3.636433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-01-22,9800.000000,10000.000000,9600.000000,9900.000000,257523,1.225953e+07,56687423,-0.133000,-9.121717e+07,7.766297e+06,...,-1.339104,-0.120869,-1.218235,-21.053490,-12.184497,-8.868993,10269.523519,-1.010101,-1.015237,87.090736
2018-01-23,9900.000000,9900.000000,9700.000000,9900.000000,184915,1.244445e+07,56872338,-0.118491,-7.554451e+07,0.000000e+00,...,-1.377588,-0.372213,-1.005375,-25.325167,-14.812631,-10.512537,10238.377416,1.020408,1.015237,88.999825
2018-01-26,9300.000000,9500.000000,8400.000000,9300.000000,277990,1.262135e+07,56594348,-0.113333,-8.858015e+07,-3.363430e+08,...,-1.871632,-0.672097,-1.199535,-27.657177,-17.381540,-10.275637,10058.950000,-6.060606,-6.252036,77.545290
